
<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to-TFlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to-TFlite.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>



# TensorFlow Lite Conversion #

<img src="https://miro.medium.com/max/1400/1*AO4WEkrSRz4Xb7qzkbqcvg.png"  alt="Visit my Blog"> 

<img src="https://lh3.googleusercontent.com/vvBAqSnXyg3h9yS0JLyVehhV-e__3NFbZ6q7Ft-rEZp-9wDTVZ49yjuYJwfa4jQZ-RVnChHMr-DDC0T_fTxVyQg3iBMD-icMQooD6A=w630-rw" alt="Visit my Blog">

Once we have our trained model, there is one final step we must take before we are ready to start testing the model. 

We have to convert the frozen graph into a TF Lite model which our inference application can use. The model is quantized, which means it is compatible with Edge TPU devices such as Google Coral.

### Install required libraries

In [1]:
%tensorflow_version 1.x
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
from constants import * 

#!git clone https://github.com/tensorflow/models.git && (cd models && git checkout f788046ca876a8820e05b0b48c1fc2e16b0955bc)
#!rsync -avq /content/models  {BASE_DIR}/ && rm -r /content/models
#%env PYTHONPATH=:{$BASE_DIR}/models/:{$BASE_DIR}/models/research/slim'


TensorFlow 1.x selected.
--2020-06-11 10:31:56--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4840 (4.7K) [text/plain]
Saving to: ‘constants.py’

constants.py        100%[===================>]   4.73K  --.-KB/s    in 0s      

2020-06-11 10:31:57 (45.5 MB/s) - ‘constants.py’ saved [4840/4840]



In [2]:
!git clone https://github.com/tensorflow/models.git && (cd models && git checkout f788046ca876a8820e05b0b48c1fc2e16b0955bc)
!rsync -avzh --progress /content/models /content/tensorflow/
!rm -rf /content/models
#!mv models /content/tensorflow/
#%cd /content && rmdir -r /content/models

import os
os.environ['PYTHONPATH'] += ':/content/tensorflow/models/research/:/content/tensorflow/models/research/slim'

Streaming output truncated to the last 5000 lines.
         59.48K 100%  196.22kB/s    0:00:00 (xfr#467, ir-chk=1018/1632)
models/research/cognitive_planning/train_supervised_active_vision.py
         18.75K 100%   61.44kB/s    0:00:00 (xfr#468, ir-chk=1017/1632)
models/research/cognitive_planning/train_supervised_active_vision.sh
          1.35K 100%    4.44kB/s    0:00:00 (xfr#469, ir-chk=1016/1632)
models/research/cognitive_planning/visualization_utils.py
         29.60K 100%   96.67kB/s    0:00:00 (xfr#470, ir-chk=1015/1632)
models/research/cognitive_planning/viz_active_vision_dataset_main.py
         13.17K 100%   42.88kB/s    0:00:00 (xfr#471, ir-chk=1014/1632)
models/research/cognitive_planning/envs/
models/research/cognitive_planning/envs/__init__.py
              0 100%    0.00kB/s    0:00:00 (xfr#472, ir-chk=1011/1632)
models/research/cognitive_planning/envs/active_vision_dataset_env.py
         41.23K 100%  115.36kB/s    0:00:00 (xfr#473, ir-chk=1010/1632)
models/research/co

In [3]:
#EDGE TPU REQUIREMENTS
#!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
#!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
#!sudo apt-get update
#!sudo apt-get install edgetpu-compiler

#!apt-get install protobuf-compiler python-tk 
#!apt-get install python-pil python-lxml 
#!pip install jupyter
#!pip install matplotlib

!pip install Cython
!pip install contextlib2
!pip install tf_slim
!pip install wget
!pip install tensorflow==1.15

import wget

     |████████████████████████████████| 358kB 3.5MB/s 
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=865d3430ea7e588e27a87ab776badec37c98116b83e3b2f1e173d6b556f01720
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
     |████████████████████████████████| 412.3MB 24kB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=0b74e31a89758938ff56189f7da23c5dda89121376ec756d79cbae1d42ed02af
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow 1.15.2
    Uninstalling tensorflow-1.15.2:
      Successfully uninstalled tensorflow-1.15.2


In [4]:
%cd {BASE_DIR}/models/research
!python setup.py build
!python setup.py install
!protoc object_detection/protos/*.proto --python_out=.

/content/tensorflow/models/research
running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/__init__.py -> build/lib/object_detection
copying object_detection/inputs.py -> build/lib/object_detection
copying object_detection/model_main.py -> build/lib/object_detection
copying object_detection/model_lib.py -> build/lib/object_detection
copying object_detection/exporter_test.py -> build/lib/object_detection
copying object_detection/exporter.py -> build/lib/object_detection
copying object_detection/eval_util.py -> build/lib/object_detection
copying object_detection/model_hparams.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph.py -> build/lib/object_detection
copying object_detection/model_lib_test.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_test.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detect

#### Download model config and save to to config directory

In [5]:
#wget.download("https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/ssd_mobilenet_v2_quantized_300x300.config",CONFIG_DIR)
wget.download(pretrained_model_key['config_url'],CONFIG_DIR)

'/content/tensorflow/workspace/Face-Mask-Detection/training/config/ssd_mobilenet_v2_quantized_300x300.config'

### Prepare dataset ###

#### Connect to Google Drive

In [6]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Copy dataset from Google Drive and extract

In [7]:
!gsutil cp {DRIVE_DEV}model_graph.zip {OBJ_DET_DIR}
!unzip -j {OBJ_DET_DIR}/model_graph.zip -d {INF_GRAPH}

!gsutil cp {DRIVE_DEV}model_checkpoints.zip {OBJ_DET_DIR}
!unzip -j {OBJ_DET_DIR}/model_checkpoints.zip -d {CKPT_DIR}


Copying file:///content/drive/My Drive/Machine Learning/Projects/Face-Mask-Detection/Dev/model_graph.zip...
-
Operation completed over 1 objects/50.6 MiB.                                     
Archive:  /content/tensorflow/workspace/model_graph.zip
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/checkpoint  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/pipeline.config  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/model.ckpt.meta  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/model.ckpt.data-00000-of-00001  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/saved_model.pb  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph

### Export inference graph

In [8]:
!python {BASE_DIR}/models/research/object_detection/export_inference_graph.py \
  --pipeline_config_path="{CONFIG_DIR}/{pretrained_model_key['name']}.config" \
  --trained_checkpoint_prefix="{CKPT_DIR}/model.ckpt-18560" \
  --output_directory="{INF_GRAPH}" \
  --add_postprocessing_op=true \
  --input_type=image_tensor \
  

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W0611 10:44:29.284205 139872719275904 module_wrapper.py:139] From /content/tensorflow/models/research/object_detection/export_inference_graph.py:133: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0611 10:44:29.293980 139872719275904 module_wrapper.py:139] From /content/tensorflow/models/research/object_detection/exporter.py:348: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0611 10:44:29.294343 139872719275904 module_wrapper.py:139] From /content/tensorflow/models/research/object_detection/exporter.py:113: The name tf.placeholder is deprecated. Please 

### Export TF Lite SSD inference graph

In [12]:
!python {BASE_DIR}/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path="{CONFIG_DIR}/{pretrained_model_key['name']}.config" \
--trained_checkpoint_prefix="{CKPT_DIR}/model.ckpt-18560" \
--output_directory="{TF_LITE_GRAPH}" \
--add_postprocessing_op=true

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W0611 10:47:56.324419 140054404839296 module_wrapper.py:139] From /content/tensorflow/models/research/object_detection/export_tflite_ssd_graph.py:133: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0611 10:47:56.328783 140054404839296 module_wrapper.py:139] From /content/tensorflow/models/research/object_detection/export_tflite_ssd_graph_lib.py:182: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0611 10:47:56.329111 140054404839296 module_wrapper.py:139] From /content/tensorflow/models/research/object_detection/export_tflite_ssd_graph_lib.py:226: The nam

### Convert graph to TF Lite

In [13]:
!tflite_convert \
--graph_def_file="{TF_LITE_GRAPH}/tflite_graph.pb" \
--output_file="{TF_LITE_MODEL}/detect.tflite" \
--output_format=TFLITE \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
--inference_type=QUANTIZED_UINT8 \
--mean_values=128 \
--std_dev_values=127 \
--change_concat_input_ranges=false \
--allow_custom_ops

2020-06-11 10:49:05.735984: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-11 10:49:05.737922: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-06-11 10:49:05.737982: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (212ebc8d9049): /proc/driver/nvidia/version does not exist
2020-06-11 10:49:05.738386: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-06-11 10:49:05.744571: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-06-11 10:49:05.744853: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x13fca00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-06-11 10:49:

## Save 

### Archive the inference graph

In [0]:
!zip -r tflite-graphv1.zip {INF_GRAPH}
!gsutil cp tflite-graphv1.zip {DRIVE_DEV}

/content/training_demo
--2020-06-05 08:59:06--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/ssd_mobilenet_v2_coco.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4766 (4.7K) [text/plain]
Saving to: ‘ssd_mobilenet_v2_coco.config’

ssd_mobilenet_v2_co 100%[===================>]   4.65K  --.-KB/s    in 0s      

2020-06-05 08:59:06 (46.5 MB/s) - ‘ssd_mobilenet_v2_coco.config’ saved [4766/4766]



In [0]:
!zip -r inf-graphv1.zip {TF_LITE_GRAPH}
!gsutil cp inf-graphv1.zip {DRIVE_DEV}

### Save archive to Google Drive

/content/training_demo/images/test
185
